In [4]:
import warnings
warnings.filterwarnings("ignore")
import webcolors
import cv2
import numpy as np
from sklearn.cluster import KMeans
from collections import defaultdict, Counter
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense



In [5]:

# Function to load and process an image
def load_image(image_path, img_size=(64, 64)):
    img = cv2.imread(image_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = cv2.resize(img, img_size)
    return img

# Function to reshape image for KMeans
def reshape_image(img):
    return img.reshape((-1, 3))

# Function to apply KMeans and return dominant colors
def extract_dominant_colors(img, num_colors=3):
    reshaped_img = reshape_image(img)
    kmeans = KMeans(n_clusters=num_colors)
    kmeans.fit(reshaped_img)
    return kmeans.cluster_centers_

# Function to create color labels
def create_color_labels(image_paths, num_colors=3):
    color_labels = defaultdict(int)
    image_labels = []
    for path in image_paths:
        img = load_image(path)
        colors = extract_dominant_colors(img, num_colors)
        colors_tuple = tuple(map(tuple, colors))
        if colors_tuple not in color_labels:
            color_labels[colors_tuple] = len(color_labels)
        image_labels.append(color_labels[colors_tuple])

    return image_labels, color_labels

# Function to prepare dataset
def prepare_dataset(image_paths, image_labels, img_size=(64, 64)):
    images = []
    labels = []
    
    for path, label in zip(image_paths, image_labels):
        img = load_image(path, img_size)
        images.append(img)
        labels.append(label)

    images = np.array(images) / 255.0  # Normalize the images
    labels = tf.keras.utils.to_categorical(labels)  # One-hot encoding of labels

    return train_test_split(images, labels, test_size=0.2)



In [6]:
# CNN Model Building
def build_cnn(input_shape, num_classes):
    model = Sequential()
    model.add(Conv2D(32, (3, 3), activation='relu', input_shape=input_shape))
    model.add(MaxPooling2D((2, 2)))
    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(MaxPooling2D((2, 2)))
    model.add(Flatten())
    model.add(Dense(64, activation='relu'))
    model.add(Dense(num_classes, activation='softmax'))

    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model


import os
def get_image_paths(folder_path, valid_extensions=(".jpg", ".jpeg", ".png")):
    image_paths = []
    for filename in os.listdir(folder_path):
        if filename.endswith(valid_extensions):
            image_paths.append(os.path.join(folder_path, filename))
    return image_paths
# Replace 'your_image_folder_path' with the path to your folder containing images
folder_path = 'small_img'  # Update this with your folder path
image_paths = get_image_paths(folder_path)

# The rest of your code for creating labels and preparing the dataset
image_labels, _ = create_color_labels(image_paths)
X_train, X_test, y_train, y_test = prepare_dataset(image_paths, image_labels)



# # Replace this with the actual image paths
# image_paths = ['upload/1 (13).jpg']  # Update this list with your image paths
# image_labels, _ = create_color_labels(image_paths)
# X_train, X_test, y_train, y_test = prepare_dataset(image_paths, image_labels)

# Build and train the CNN
num_classes = y_train.shape[1]
model = build_cnn(X_train.shape[1:], num_classes)
history = model.fit(X_train, y_train, epochs=20, validation_split=0.2)






Epoch 1/20


4/4 [==============================] - 2s 149ms/step - loss: 5.4336 - accuracy: 0.0000e+00 - val_loss: 5.3710 - val_accuracy: 0.0000e+00
Epoch 2/20
4/4 [==============================] - 0s 62ms/step - loss: 5.2578 - accuracy: 0.0000e+00 - val_loss: 5.3406 - val_accuracy: 0.0000e+00
Epoch 3/20
4/4 [==============================] - 0s 68ms/step - loss: 5.2278 - accuracy: 0.0234 - val_loss: 5.3764 - val_accuracy: 0.0000e+00
Epoch 4/20
4/4 [==============================] - 0s 63ms/step - loss: 5.1524 - accuracy: 0.0234 - val_loss: 5.5633 - val_accuracy: 0.0000e+00
Epoch 5/20
4/4 [==============================] - 0s 67ms/step - loss: 5.0198 - accuracy: 0.0312 - val_loss: 5.9255 - val_accuracy: 0.0000e+00
Epoch 6/20
4/4 [==============================] - 0s 68ms/step - loss: 4.8776 - accuracy: 0.0625 - val_loss: 6.0921 - val_accuracy: 0.0000e+00
Epoch 7/20
4/4 [==============================] - 0s 91ms/step - loss: 4.6770 - accuracy: 0.0859 - val_loss: 6.3102 - val_accura

In [7]:
# Access the history to get training accuracy
training_accuracy = history.history['accuracy']
print("Training Accuracy: ", training_accuracy[-1])
model.save('model.h5')

Training Accuracy:  1.0


In [8]:
emotions = {
    'black': ['Powerful', 'sophisticated', 'edgy'],
    'firebrick': ['Passionate', 'aggressive', 'important'],
    'gold': ['Opulent', 'traditional', 'prestigious'],
    'silver': ['Sleek', 'graceful', 'futuristic'],
    'turquoise': ['Refreshing', 'tranquil', 'creative'],
    'lavender': ['Delicate', 'graceful', 'nostalgic'],
    'beige': ['Simplistic', 'dependable', 'conservative'],
    'palevioletred': ['Dynamic', 'bold', 'passionate'],
    'cornflowerblue': ['Imaginative', 'spirited', 'unique'],
    'darkslategray': ['Natural', 'peaceful', 'enduring'],
    'rosybrown': ['Warm', 'inviting', 'vibrant'],
    'darkolivegreen': ['Professional', 'reliable', 'authoritative'],
    'olivedrab': ['Energetic', 'lively', 'fresh'],
    'lightgray': ['Solid', 'professional', 'mature'],
    'steelblue': ['Fresh', 'cool', 'youthful'],
    'darkgray': ['Deep', 'wise', 'thoughtful'],
    'dimgray': ['Lush', 'vibrant', 'sophisticated'],
    'cadetblue': ['Earthy', 'warm', 'enduring'],
    'gray': ['Soft', 'friendly', 'approachable'],
    'slategray': ['Refreshing', 'serene', 'youthful']
}

In [9]:
import cv2
import numpy as np
from sklearn.cluster import KMeans
from tensorflow.keras.models import load_model

import webcolors

def closest_color(requested_color):
    min_colors = {}
    for key, name in webcolors.CSS3_HEX_TO_NAMES.items():
        r_c, g_c, b_c = webcolors.hex_to_rgb(key)
        rd = (r_c - requested_color[0]) ** 2
        gd = (g_c - requested_color[1]) ** 2
        bd = (b_c - requested_color[2]) ** 2
        min_colors[(rd + gd + bd)] = name
    return min_colors[min(min_colors.keys())]

def get_color_name(hex_value):
    try:
        color_name = webcolors.hex_to_name(hex_value)
    except ValueError:
        rgb_value = webcolors.hex_to_rgb(hex_value)
        color_name = closest_color(rgb_value)
    return color_name


def load_and_preprocess_image(image_path, img_size=(64, 64)):
    # Load the image
    img = cv2.imread(image_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    
    # Resize and normalize the image
    img = cv2.resize(img, img_size)
    img = img / 255.0
    
    return np.expand_dims(img, axis=0)

def find_dominant_color(image, k=1):
    # Reshape the image and apply KMeans clustering
    reshaped_img = image.reshape((-1, 3))
    kmeans = KMeans(n_clusters=k)
    kmeans.fit(reshaped_img)

    # Get the dominant color
    dominant_color = kmeans.cluster_centers_[0].astype(int)

    # Convert the dominant color to hex
    return "#{:02x}{:02x}{:02x}".format(dominant_color[0], dominant_color[1], dominant_color[2])

# Load your trained model
loaded_model = load_model('model.h5')

# Path to the new image you want to predict on
new_image_path = 'upload/1 (10).jpg'

# Load and preprocess the image
preprocessed_image = load_and_preprocess_image(new_image_path)

# Predict the class of the image
predictions = loaded_model.predict(preprocessed_image)
predicted_class = np.argmax(predictions, axis=1)

# Load the image again in its original size for color extraction
original_image = cv2.imread(new_image_path)
original_image = cv2.cvtColor(original_image, cv2.COLOR_BGR2RGB)

# Find the dominant color
dominant_hex_color = find_dominant_color(original_image)
color=get_color_name(dominant_hex_color)
main_emotion = emotions[color]
# Output
# print("Predicted Class:", predicted_class[0])
print("Dominant Hex Color:", dominant_hex_color)
print("Predicted Color :",color) 
print("Emotions:",main_emotion)

1/1 [==============================] - 0s 113ms/step
Dominant Hex Color: #d9d8d4
Predicted Color : lightgray
Emotions: ['Solid', 'professional', 'mature']


In [10]:
import cv2
import numpy as np
from tensorflow.keras.models import load_model

# Assuming the other functions (closest_color, get_color_name, find_dominant_color) are defined as before

def process_video(video_path, model_path, img_size=(64, 64)):
    # Load the trained model
    loaded_model = load_model(model_path)

    # Open the video file
    cap = cv2.VideoCapture(video_path)

    while cap.isOpened():
        ret, frame = cap.read()

        if not ret:
            break

        # Preprocess the frame (resize, normalize)
        frame_processed = cv2.resize(frame, img_size)
        frame_processed = frame_processed / 255.0
        frame_processed = np.expand_dims(frame_processed, axis=0)

        # Predict the class for the current frame
        predictions = loaded_model.predict(frame_processed)
        predicted_class = np.argmax(predictions, axis=1)

        # Convert frame to RGB and find the dominant color
        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        dominant_hex_color = find_dominant_color(frame_rgb)
        color = get_color_name(dominant_hex_color)
        main_emotion = emotions[color]
        return dominant_hex_color,color,main_emotion
        # Process the results as needed (e.g., display, store, aggregate)

    # Release the video capture object
    cap.release()

# Example usage
video_path = 'upload/v2.mp4'
model_path = 'model.h5'
dominant_hex_color,color,main_emotion = process_video(video_path, model_path)
print(dominant_hex_color)
print(color)
print(main_emotion)


1/1 [==============================] - 0s 99ms/step
#201e1a
black
['Powerful', 'sophisticated', 'edgy']
